# 📦 Caso de Prueba de Aplicación: Prueba CRUD en MySQL

Para completar la comparación entre una máquina virtual (VM) y un contenedor Docker, se diseñó una prueba de rendimiento de una aplicación práctica basada en un servidor **MySQL**.

Este caso de prueba se centra en medir cómo se comportan ambos entornos al ejecutar operaciones masivas sobre una base de datos, simulando una carga de trabajo realista.

---

## 🧪 Descripción de la Prueba

Se desarrolló un script en Python que utiliza la librería **Faker** para generar datos falsos. Este script realiza una serie de operaciones CRUD (Crear, Leer, Actualizar, Eliminar) en bloque sobre una tabla MySQL.

Las métricas medidas son las siguientes:

1. **Tiempo de inserción masiva**  
   Inserción de 10,000 registros generados automáticamente.

2. **Tiempo de consulta total (SELECT)**  
   Consulta de todos los registros para evaluar la velocidad de lectura masiva.

3. **Tiempo de actualización de salarios (UPDATE)**  
   Actualización en bloque de todos los registros para simular cambios masivos (ej. aumento de salario).

4. **Tiempo de eliminación completa (DELETE)**  
   Eliminación de todos los registros para simular una limpieza masiva de la base de datos.

---

## 🔨 Herramientas y Tecnologías

- **Python 3**
- Librerías:
  - `mysql-connector-python`
  - `faker`
  - `psutil` (para monitoreo de recursos)
- **MySQL 8.0**
- Entornos probados:
  - Docker (`mysql_benchmark` + `mysql_tester`)
  - Máquina Virtual (VirtualBox)



## 📊 Objetivo de la Comparación

- Evaluar y comparar:
  - **Velocidad de operaciones**: tiempos medidos en segundos.
  - **Uso de recursos**: CPU y memoria promedio durante las operaciones.
- Determinar:
  - ¿Cuál entorno es más eficiente para cargas de trabajo intensivas?
  - ¿Dónde se observa mejor manejo de recursos bajo carga masiva?


## ✅ Conclusiones esperadas

Esta prueba permite entender:
- Cómo se comportan los contenedores Docker frente a las máquinas virtuales en escenarios reales.
- Qué ventajas ofrece cada entorno según el tipo de carga (en este caso, operaciones CRUD intensivas sobre base de datos).



In [ ]:
#Imports y configuración inicial
import mysql.connector
import time
import os
import psutil
import threading
from faker import Faker
from mysql.connector import Error

fake = Faker()
NUM_REGISTROS = 10000

# Variables para monitoreo
cpu_samples = []
mem_samples = []
monitoreo_activo = True


In [ ]:
#Función de monitoreo y arranque del hilo
def monitorear_recursos():
    """Toma muestras cada segundo del uso de CPU y RAM."""
    while monitoreo_activo:
        cpu_samples.append(psutil.cpu_percent(interval=1))
        mem_samples.append(psutil.virtual_memory().used / (1024 ** 2))  # MB

# Iniciar monitoreo en segundo plano
monitor_thread = threading.Thread(target=monitorear_recursos)
monitor_thread.start()


In [ ]:
#Conexión a MySQL
MAX_INTENTOS = 10
intento = 1

while intento <= MAX_INTENTOS:
    try:
        conn = mysql.connector.connect(
            host=os.environ.get("MYSQL_HOST", "localhost"),
            user=os.environ.get("MYSQL_USER", "root"),
            password=os.environ.get("MYSQL_PASSWORD", "password"),
            database=os.environ.get("MYSQL_DATABASE", "persona")
        )
        if conn.is_connected():
            print(f"✅ Conectado a MySQL en el intento {intento}")
            break
    except Error as e:
        print(f"❌ Intento {intento}: esperando conexión a MySQL... ({e})")
        time.sleep(3)
        intento += 1
else:
    monitoreo_activo = False
    raise Exception("⛔ No se pudo conectar a MySQL después de varios intentos.")

cursor = conn.cursor()


In [ ]:
#Crear tabla
print("🎯 Creando tabla tbl_persona...")
cursor.execute("DROP TABLE IF EXISTS tbl_persona")
cursor.execute("""
    CREATE TABLE tbl_persona (
        id INT AUTO_INCREMENT PRIMARY KEY,
        nombre VARCHAR(450),
        apellido VARCHAR(450),
        email VARCHAR(450),
        telefono VARCHAR(450),
        direccion TEXT,
        ciudad VARCHAR(450),
        pais VARCHAR(450),
        fecha_nacimiento DATE,
        empresa VARCHAR(450),
        puesto VARCHAR(450),
        salario DECIMAL(10,2),
        creado_en TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")


In [ ]:
#Insertar datos
print("🧪 Insertando datos...")
inicio = time.time()
query = """
    INSERT INTO tbl_persona (nombre, apellido, email, telefono, direccion, ciudad, pais,
                             fecha_nacimiento, empresa, puesto, salario)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
datos = [
    (
        fake.first_name(),
        fake.last_name(),
        fake.email(),
        fake.phone_number()[:450],
        fake.address(),
        fake.city()[:450],
        fake.country()[:450],
        fake.date_of_birth(minimum_age=18, maximum_age=65),
        fake.company()[:450],
        fake.job()[:450],
        round(fake.random_number(digits=5) / 10, 2)
    )
    for _ in range(NUM_REGISTROS)
]

cursor.executemany(query, datos)
conn.commit()
fin = time.time()
print(f"✅ Inserción de {NUM_REGISTROS} registros en {fin - inicio:.2f} segundos")


In [ ]:
print("🔍 Consultando todos los registros...")
inicio = time.time()
cursor.execute("SELECT * FROM tbl_persona")
result = cursor.fetchall()
fin = time.time()
print(f"✅ SELECT total: {len(result)} registros en {fin - inicio:.2f} segundos")


In [ ]:
#UPDATE masivo
print("✏️ Actualizando todos los salarios (aumentando 10%)...")
inicio = time.time()
cursor.execute("UPDATE tbl_persona SET salario = salario * 1.10")
conn.commit()
fin = time.time()
print(f"✅ UPDATE total completado en {fin - inicio:.2f} segundos")


In [ ]:
print("🗑️ Eliminando todos los registros...")
inicio = time.time()
cursor.execute("DELETE FROM tbl_persona")
conn.commit()
fin = time.time()
print(f"✅ DELETE total completado en {fin - inicio:.2f} segundos")


In [ ]:
#Calcular promedios y mostrar resumen
monitoreo_activo = False
monitor_thread.join()

cpu_avg = sum(cpu_samples) / len(cpu_samples) if cpu_samples else 0
mem_avg = sum(mem_samples) / len(mem_samples) if mem_samples else 0

print(f"\n📊 Consumo promedio durante la prueba:")
print(f"   CPU: {cpu_avg:.2f}%")
print(f"   RAM: {mem_avg:.2f} MB")
#cerrar la conexion cursor.close()
conn.close()
print("🏁 Prueba de rendimiento completada.")